# Importing a CSV file into an Azure MySQL database:

_Author: Matěj Srna_
***
<img src="https://static.wixstatic.com/media/2d21b4_ca5a171b85cd4e839032435655ed00ec~mv2.png" style="width:350px; height:350px; margin-left: auto; margin-right: auto" />

 
> _**Disclaimer: This import to the database is only used for the educational purposes**_

***
This part of the code consists of several parts:
 * first step is to importing the CSV file into the pandas DataFrame,
 * second step is to clean the table column names and clean it from unnecessary symbols,
 * third step is to create the table if it does not exist in SQL - statement,
 * last step is to upload the data into the our database.
 
 ***

#### Content:
1. [Module imports](#Module-imports:)
2. [Getting the file path and name](#Getting-the-file-path-and-name:)
3. [Reading the data from CSV File which had been cleaned](#Reading-the-data-from-CSV-File-which-had-been-cleaned:)
4. [Preview of the MySQL query to create the table with all necessary columns](#Preview-of-the-MySQL-query-to-create-the-table-with-all-necessary-columns: )
5. [Connecting to the MySQL database](#Connecting-to-the-MySQL-database:)
5. [Saving the clean dataset to CSV for further analysis](#Saving-the-clean-dataset-to-CSV-for-further-analysis:)

### Module imports:

In [4]:
import mysql.connector
from mysql.connector import errorcode
import psycopg2
import connection_db

from time import sleep
import sys
import os
import numpy as np
import pandas as pd
from tqdm import tqdm

### Getting the file path and name:

***
Getting the directory to the file which will be imported to the database and it's name. To get the name of the file it's necessary to access the directory where the file is stored. After the accessing, path is return to the previous directory.
***

In [5]:
#getting the path to the file with os module with getcwd()
try:
    os.getcwd()
    os.chdir("data_import")
except:
    pass
file_name = os.listdir()
file_path = os.getcwd()
print(file_name)
os.chdir("..")

['clean_data.csv']


In [6]:
##os.getcwd()

### Reading the data from CSV File which had been cleaned:

***
Importing data from the clean CSV file into a dataset with file path and filename retrieved by code.
***

In [7]:
#importing data with pandas read_csv combining the file path and filename retrieved above
data = pd.read_csv(file_path+"\\"+"".join(file_name))
data.head()

,name,company_name,ago,contract,location,date,description,link
0,Data Scientist,SEITEQ,Před 2 týdny,Plný úvazek,"Hlavní město Praha, Česko",2022-05-15,Analyzovat data pro obchodní potřeby našich zá...,https://cz.linkedin.com/jobs/view/data-scienti...
1,Data Scientist,Merkle DACH,Před 3 týdny,Střední služební věk,"Hlavní město Praha, Česko",2022-05-15,Data ScientistWe Dream. We Do. We Deliver.As a...,https://cz.linkedin.com/jobs/view/data-scienti...
2,Senior Data Scientist,Hays,Před 1 týdnem,Střední služební věk,"Hlavní město Praha, Česko",2022-05-15,"Our customer, technology company has built a s...",https://cz.linkedin.com/jobs/view/senior-data-...
3,Senior Data Scientist,Custom Ink,Před 3 týdny,Střední služební věk,"Hlavní město Praha, Česko",2022-05-15,Custom Ink is a fast growing e-commerce and ma...,https://cz.linkedin.com/jobs/view/senior-data-...
4,Lead Data Scientist,Merkle DACH,Před 1 týdnem,Střední služební věk,"Hlavní město Praha, Česko",2022-05-15,Lead Data ScientistWe Dream. We Do. We Deliver...,https://cz.linkedin.com/jobs/view/lead-data-sc...


<div class="alert alert-block alert-info">
<b>Tip:</b> Automatically loads any CSV file which is stored in particular directory.</div>

***
In the code below I prepare the name of the table which will be used in the creation of the MySQL table. Based on the naming conventions for MySQL table the characters must be allowed. Mostly the names cannot consist of symbols and quotation marks. Therefore, I used replace function to remove all the characters which are not allowed. I also do this process with the column names for this table.
***

In [8]:
#cleaning the CSV file name for the database import
    # lower cases letter
    # to remove all the white spaces
    # replace -, /, \\, # with _
    
file_name = "".join(file_name).replace(".csv","_jobs")

clean_tbl_name = file_name.lower().replace(" ", "_").replace("?", "").replace("-", "_").replace(r"/","_").replace("\\","_").replace("%","").replace(")","").replace(r"(","").replace("?","")
print(clean_tbl_name)

clean_data_jobs


In [9]:
#cleaning the table column name for the database import
    # lower cases letter
    # to remove all the white spaces
    # replace -, /, \\, # with _
    
data.columns = [x.lower().replace(" ", "_").replace("?", "").replace("-", "_").replace(r"/","_").replace("\\","_").replace("%","").replace(")","").replace(r"(","").replace("?","") for x in data.columns]

print(data.columns)

Index(['name', 'company_name', 'ago', 'contract', 'location', 'date',
       'description', 'link'],
      dtype='object')


### Preview of the MySQL query to create the table with all necessary columns: 

```mysql
    CREATE TABLE IF NOT EXISTS jobs_summary_data = (
        name           varchar,
        company_name   varchar,
        ago            varchar,
        contract       varchar,
        location       varchar,
        date           date,
        description    text CHARACTER SET utf8,
        link           varchar,   
    )
```

In [10]:
data = data.astype({"date": "datetime64[ns]"})
data = data.astype({"description": "string"})

In [11]:
data.dtypes

name                    object
company_name            object
ago                     object
contract                object
location                object
date            datetime64[ns]
description             string
link                    object
dtype: object

***
In the code below, I change the DataFrame's data type into the MySQL data types. Therefore, I can make a string with all the column names with correct data types. This string is used later on to create automatically the MySQL table.
***

In [12]:
replacement = {
    "object": "varchar(500)",
    "float64": "float",
    "int64": "int",
    "datetime64[ns]": "timestamp(0)",
    "timesdelta64[ns]": "varchar(500)",
    "string": "text CHARACTER SET UTF8MB4"
}

In [13]:
con_str = ", ".join("{} {}".format(n, d) for (n, d) in zip (data.columns, data.dtypes.replace(replacement)))
con_str

'name varchar(500), company_name varchar(500), ago varchar(500), contract varchar(500), location varchar(500), date timestamp(0), description text CHARACTER SET UTF8MB4, link varchar(500)'

<div class="alert alert-block alert-success">
<b>Success:</b> This string contains the transformed names of the columns in DataFrame and combines them with MySQL data types. This string is used for the creation of the MySQL table in the code below.
</div>

### Connecting to the MySQL database: 

***
For storing of this data I chose the Azure MySQL database, which is cloud relation database service in the Microsoft cloud. To connect to my database I used mysql.connector module/library.
***

In [14]:
config = {
            "user": "msrna",
            "password": connection_db.password,
            "database": "jobpostingdata",
            "host": "mysqlms12.mysql.database.azure.com",
            "port": "3306",                              
}


try:
    conn = mysql.connector.connect(**config)
    print("Connection established")
except mysql.connector.Error as err:
    if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
        print("Something is wrong with the user name or password")
    elif err.errno == errorcode.ER_BAD_DB_ERROR:
        print("Database does not exist")
    else:
        print(err)
else:
    cursor = conn.cursor()

Connection established


In [15]:
# Drop previous table of same name if one exists
cursor.execute(f"DROP TABLE IF EXISTS {clean_tbl_name};")
print("Finished dropping table (if existed).")

Finished dropping table (if existed).


In [16]:
# Create table
cursor.execute(f"CREATE TABLE IF NOT EXISTS {clean_tbl_name} ({con_str});")
print("Finished creating table.")

Finished creating table.


<div class="alert alert-block alert-success">
<b>Success:</b> The string created above contains the transformed names of the columns in DataFrame combined with MySQL data types. With this string, the MySQL table is created.
</div>

***
In the code below, I combined the MySQL query string with iterating in the data frame containing all the web scraped data. Therefore, I am able to load all the data into the MySQL database. I also used the tqdm progress bar to show how the loading progresses.
***

In [17]:
num_rows = len(data.index)
index = 0

try:
    with tqdm(total=num_rows) as pbar:
        pbar.set_description("Uploading %s of rows into database" % num_rows)
        for (row, rs) in data.iterrows():
            index += 1
            name = rs[0]
            company_name = rs[1].replace("'","").replace("’","")
            ago = rs[2]
            contract = rs[3]
            location = rs[4]
            date = str(rs[5])
            description = rs[6].replace("'"," ").replace("$","").replace("’","")
            link =  rs[7]
            query=f"INSERT IGNORE INTO {clean_tbl_name} \
        VALUES ("+"'"+name+"'"+", "+"'"+company_name+"'"+", "+"'"+ago+"'"+","+"'"+contract+"'"+"\
        , "+"'"+location+"'"+", "+"'"+date+"'"+", "+"'"+description+"'"+", "+"'"+link+"'"+");"
            cursor.execute(query)
            pbar.update(1)
            
except:
    pass
conn.commit()
#print(query)

Uploading 1725 of rows into database: 100%|████████████████████████████████████████| 1725/1725 [00:51<00:00, 33.41it/s]


<div class="alert alert-block alert-info">
<b>Tip:</b> To see the progress of data loading into the database I used tqdm bar to show progress.</div>

In [18]:
# Cleanup
try:
    conn.commit()
except:
    pass
try:
    cursor.close()
    print("Cursor closed")
except:
    pass
if conn.is_connected():
    conn.close()
    print("Database closed")
print("Done.")

Cursor closed
Database closed
Done.


### Accessing the loaded data from the database:

***
To varify if the data was successfuly loaded, I access the database and select the data using the MySQL query:
```mysql
SELECT NAME, COUNT(NAME) FROM clean_data_jobs GROUP BY NAME ORDER BY COUNT(NAME) DESC;

```

***

In [19]:
config = {
            "user": "msrna",
            "password": connection_db.password,
            "database": "jobpostingdata",
            "host": "mysqlms12.mysql.database.azure.com",
            "port": "3306",                              
}


try:
    conn = mysql.connector.connect(**config)
    print("Connection established")
except mysql.connector.Error as err:
    if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
        print("Something is wrong with the user name or password")
    elif err.errno == errorcode.ER_BAD_DB_ERROR:
        print("Database does not exist")
    else:
        print(err)
else:
    cursor = conn.cursor()

Connection established


In [20]:
cursor.execute("SELECT NAME, COUNT(NAME) FROM clean_data_jobs GROUP BY NAME ORDER BY COUNT(NAME) DESC;")
names = cursor.fetchall()

***
Data is loaded by MySQL query using the cursor execute and function fetchall into the variable name. To see the data more clearly I put it into the pandas DataFrame with specified column names.
***

In [23]:
names_d = pd.DataFrame(names, columns=["Names", "Count"])
names_d

,Names,Count
0,Data Scientist,1298
1,Senior Data Scientist,156
2,Big Data Engineer,107
3,Junior Data Scientist,57
4,Lead Data Scientist,54
5,Data Analyst,24
6,AI Engineer,12
7,Senior Data Engineer,4
8,Staff Data Scientist,4
9,Data Engineer,3


<div class="alert alert-block alert-success">
<b>Success:</b> The data was successfuly loaded and visualized in table.
</div>

In [22]:
try:
    cursor.close()
    print("Cursor closed")
except:
    pass
if conn.is_connected():
    conn.close()
    print("Database closed")

Cursor closed
Database closed


<br>
<br>
<span style="color:green"><p style="text-align:right; font-style: italic"><b>Matěj Srna</b></p></span>
<hr></hr>
<em>My links:</em>
<br>
<table style="float:left; width: 250px; border-collapse: separate;">
<thead>
<tr><th><a href="https://www.linkedin.com/in/matejsrna" target="_blank" rel="noopener noreferrer"><img src="https://static.wixstatic.com/media/2d21b4_80567ee7301a4a50ada13620eaa1028d~mv2.png" style="width:48px; height:48px"/></a></th><th><a href="https://github.com/srnamaty" target="_blank" rel="noopener noreferrer"><img src="https://static.wixstatic.com/media/2d21b4_1d247a3f36384cd8b0eecf23b2010fae~mv2.png" style="width:58px; height:58px" /></a></th><th><a href="https://www.srnamatej.com" target="_blank" rel="noopener noreferrer"><img src="https://static.wixstatic.com/media/2d21b4_17665bc36b10443c8446ea225ce8f748~mv2.png" style="width:58px; height:58px" /></a></th></tr>
</thead>
</table>